In [1]:
import os
import pybullet as p
import json
import math
import numpy as np
import imageio

pybullet build time: Jan 28 2022 20:17:22


In [2]:
from grasp_pc import set_pose_dofs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
dataset_dir = "/root/data/output_dataset"

refined_dir = "refined_grasps"

# object_model_name = "003_cracker_box_google_16k_textured_scale_1000"
object_model_name = "010_potted_meat_can_google_16k_textured_scale_1000"

gripper_name = "fetch_gripper"
# gripper_name = "Barrett"
# gripper_name = "HumanHand"
# gripper_name = "panda_grasp"

grasps_fpath = os.path.join(
    dataset_dir,
    refined_dir, f"refined_{object_model_name}-{gripper_name}.json")

urdf_dir = '../prepare-graspit-urdf/urdf/'


# images_dir = './images/' + gripper_name
images_dir = os.path.join(dataset_dir, object_model_name, "images", gripper_name)

if not os.path.isdir(images_dir):
    os.makedirs(images_dir)

In [4]:
# Camera related stuff

width = 400
height = 400
fov = 60
aspect = width / height
near = 0.02
far = 1

view_matrix = p.computeViewMatrix([0, 0, 0.5], [0, 0, 0], [1, 0, 0])
projection_matrix = p.computeProjectionMatrixFOV(fov, aspect, near, far)

cam_target_pos = [-0.97, 0.21, -0.33]
distance = 1.80
cam_pitch = -23.80
cam_yaw = 74.0
cam_roll = 0
up_axis = 2 # 1 for Y-axis, 2 for Z-axis

view_matrix_2 = p.computeViewMatrixFromYawPitchRoll(
    cam_target_pos,
    distance,
    cam_yaw,
    cam_pitch,
    cam_roll,
    2)


In [5]:
# Gripper URDF Loading part

if gripper_name == "fetch_gripper":
    gripper_urdf = "fetch_gripper/fetch_gripper.urdf"
elif gripper_name == "Barrett":
    gripper_urdf = "Barrett/Barrett.urdf"
elif gripper_name == "HumanHand":
    gripper_urdf = "HumanHand/HumanHand.urdf"
elif gripper_name == "Allegro":
    gripper_urdf = "Allegro/allegro_hand_description_right.urdf"
elif gripper_name == "panda_grasp":
    gripper_urdf = "franka_panda/panda_arm.urdf"

gripper_urdf_file = os.path.join(urdf_dir, gripper_urdf)

In [6]:
p.connect(p.GUI) # with GUI, useful to see final result
# pybullet.connect(pybullet.DIRECT) # without GUI, useful when coding and trying out pybullet functions

0

In [7]:
# grasps_fpath = "../output_grasps/003_cracker_box_google_16k_textured_scale_1000-HumanHand.json"
# grasps_fpath = "../output_grasps/003_cracker_box_google_16k_textured_scale_1000-Barrett.json"
# grasps_fpath = "../output_grasps/003_cracker_box_google_16k_textured_scale_1000-fetch_gripper.json"
# grasps_fpath = "../output_grasps_old/mano/glass.json"
# grasps_fpath = "/root/data/output_grasps/sampled-num100_003_cracker_box_google_16k_textured_scale_1000.0-fetch_gripper.json"

# IGNORE ABOVE commnted out code
# grasps_fpath = "../../output_dataset/initial_grasps/003_cracker_box_google_16k_textured_scale_1000-fetch_gripper.json"
grasps_fpath = "../../output_dataset/initial_grasps/010_potted_meat_can_google_16k_textured_scale_1000-fetch_gripper.json"
with open(grasps_fpath, "r") as gf:
    grasp_data = json.load(gf)

In [8]:
# f1 = "/output_dataset/refined_grasps/refined_003_cracker_box_google_16k_textured_scale_1000-fetch_gripper.json"
# f2 = "output_grasps/refined_grasps/refined_003_cracker_box_google_16k_textured_scale_1000-fetch_gripper.json"

In [9]:
# Load the Object
# object_urdf_file = os.path.join(args.obj_urdf_dir, model + ".urdf")
# obj_id = p.loadURDF("./003_cracker_box_custom.urdf")
# obj_id = p.loadURDF("./ycb-urdfs/003_cracker_box_google_16k_textured_scale_1000.urdf")
obj_id = p.loadURDF("./ycb-urdfs/010_potted_meat_can_google_16k_textured_scale_1000.urdf")

In [10]:
grasps = grasp_data['grasps']
robo_id = p.loadURDF(gripper_urdf_file)
for idx, info in enumerate(grasps):
    full_pose = info['pose']
    pos = full_pose[:3]
    orn = full_pose[3:]
    dofs = info['dofs']
    # tmp = p.loadURDF(gripper_urdf_file, basePosition=pos, baseOrientation=orn)
    set_pose_dofs(gripper_name, robo_id, full_pose, dofs)
    # w,h,img_rgb,img_dep,img_seg = p.getCameraImage(width, height, view_matrix_2, projection_matrix)
    # img = np.reshape(img_rgb, (w,h,4))
    img_fname = f'{gripper_name}-{object_model_name[:-20]}-refined_gnum_{idx}.png'
    # imageio.imwrite(os.path.join(images_dir, img_fname), img)
    # img_fname = f'img_graspnum_{idx}.png'

In [11]:
p.isNumpyEnabled()

1

In [12]:
test = 'ninad\
adad'

In [13]:
# #### For GUI Mode
# # w,h,img_rgb,img_dep,img_seg = pybullet.getCameraImage(900,900)
# w,h,img_rgb,img_dep,img_seg = p.getCameraImage(width, height, view_matrix_2, projection_matrix)
# img = np.reshape(img_rgb, (w,h,4))
# imageio.imwrite(f'./images/sample_gripper_viz_refined_{object_model_name}-{gripper_name}.png', img)

In [14]:
#### For Direct MODE
# viewMat = pybullet.computeViewMatrixFromYawPitchRoll([-0.38,0.32,-0.34], 1.0, 50, -35, 0, 1)
# w,h,img_rgb,img_dep,img_seg = pybullet.getCameraImage(900,900, viewMat)
# img = np.reshape(img_rgb, (w,h,4))
# imageio.imwrite('./sample_viz_1.png', img)